In [1]:
import pandas as pd
import glob

In [16]:
lista_directorios = [ str(anio) for anio in range(2010,2022) ]

In [66]:
archivos = ['A_*_h', 'A_*_m', 'CI_*_h','CI_*_m']

tabla_resultados =[]

for archivo in archivos:

    bloques_a_h = []

    for dir in lista_directorios:
        path = dir + '/' + archivo.replace('*', f'{dir}') + '.xlsx'
        df = pd.read_excel(path)
        # make edad index
        df = df.set_index('edad')
        # keep only Total column
        df = df[['Total']]
        # change to int 
        df = df.astype(int)
        # rename column
        df = df.rename(columns={'Total': 'Total'+dir})
        bloques_a_h.append(df)

    # join all dataframes
    df = pd.concat(bloques_a_h, axis=1)
    # fill NaN with 0
    df = df.fillna(0)
    # change to int
    df = df.astype(int)

    # select df where index is less than 99
    df_first = pd.DataFrame(df[df.index <= 99])
    # select df where index is greater than 99
    df_second = pd.DataFrame(df[df.index >= 100])

    datos_agrupados = {}
    # sum by each column
    for col in df_second.columns:
        datos_agrupados[col] = df_second[col].sum()

    # add datos agrupados to df_first in index 100
    df_first.loc[100] = datos_agrupados

    tabla_resultados.append(df_first)

In [68]:
# save in excel file by sheet
with pd.ExcelWriter('Consolidado_global.xlsx') as writer:
    for i, df in enumerate(tabla_resultados):
        df.to_excel(writer, sheet_name=archivos[i].replace('_*_', '_'))